<a href="https://colab.research.google.com/github/Storm00212/JARVIS/blob/main/JARVIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install -q --upgrade \
  langchain \
  langchain-community \
  langchain-text-splitters \
  sentence-transformers \
  faiss-cpu \
  pypdf \
  python-docx \
  python-pptx \
  tqdm

In [19]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
import os

BASE_DIR = "/content/drive/MyDrive/jarvis-ai"
DATA_DIR = f"{BASE_DIR}/data/raw"
VECTORSTORE_DIR = f"{BASE_DIR}/vectorstore/faiss_index"

os.makedirs(VECTORSTORE_DIR, exist_ok=True)

print("Data dir exists:", os.path.exists(DATA_DIR))


Data dir exists: True


In [21]:
from langchain_community.document_loaders import (
    PyPDFLoader,
    Docx2txtLoader,
    UnstructuredPowerPointLoader
)
from langchain_core.documents import Document
from tqdm import tqdm

In [22]:
def load_documents(folder_path):
    documents = []

    for file in tqdm(os.listdir(folder_path)):
        path = os.path.join(folder_path, file)

        try:
            if file.lower().endswith(".pdf"):
                loader = PyPDFLoader(path)

            elif file.lower().endswith(".docx"):
                loader = Docx2txtLoader(path)

            elif file.lower().endswith(".pptx"):
                loader = UnstructuredPowerPointLoader(path)

            else:
                continue

            documents.extend(loader.load())

        except Exception as e:
            print(f"Failed to load {file}: {e}")

    return documents


In [23]:
docs = load_documents(DATA_DIR)
print(f"Loaded {len(docs)} raw documents/pages")


 34%|███▍      | 28/82 [04:27<20:31, 22.80s/it]

Failed to load EEE 3207 ELECTRICAL MACHINES 2 (2).pptx: unstructured package not found, please install it with `pip install unstructured`


100%|██████████| 82/82 [09:15<00:00,  6.77s/it]

Loaded 11843 raw documents/pages


In [25]:
# chunking
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,        # safe for LLMs
    chunk_overlap=200       # preserves context
)

chunks = text_splitter.split_documents(docs)
print(f"Created {len(chunks)} text chunks")

Created 26187 text chunks


In [27]:
from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

/tmp/ipython-input-1352497980.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]